In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# !pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import shutil
import sys
import pickle

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer, RobertaModel

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
# dataset = pd.read_csv('drive/MyDrive/NLP_Final_Project/dataset/quotes.csv', delimiter=',')
# nRow, nCol = dataset.shape
# print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
# # Null Value check
# print(dataset.isnull().sum())
# dataset.dropna(inplace=True)
# dataset.quote.drop_duplicates(inplace=True)

# # Remove commas from categories. ['Life, Motivation']
# dataset['category'] = dataset['category'].str.replace(', ', ' ')

# # Strip extra spaces in category
# dataset['category'] = dataset['category'].str.strip()
# dataset['quote'] = dataset['quote'].str.strip()
# dataset['author'] = dataset['author'].str.strip()

# # Convert upper case to lower case
# dataset['category'] = dataset['category'].str.lower()
# dataset['quote'] = dataset['quote'].str.lower() ## comment if needed
# dataset['author'] = dataset['author'].str.lower() ## comment if needed

In [ ]:
# dataset.head(10)

In [ ]:
# # Creating a dictionary to store count for all tags.

# tags_dict = {}

# for row in dataset.itertuples():
#     for cat in row.category.split(' '):
#         if cat=='':
#             continue
#         tags_dict[cat.lower()] = tags_dict.get(cat.lower(), 0) + 1

# sorted_tags_dict = dict(sorted(tags_dict.items(), key=lambda x: x[1], reverse=True))

In [ ]:
# # Remove all the tags whose count is below the specified threshold

# threshold = 1500

# tags_to_remove = [key for key, value in sorted_tags_dict.items() if value < threshold]

# # Remove items from the dictionary
# for key in tags_to_remove:
#     del sorted_tags_dict[key]

# print(f'After removing less frequent tags: there are {len(sorted_tags_dict)} types of different tags')

# # Null Value check
# dataset.dropna(inplace=True)

In [ ]:
# # List of tags to keep
# top_k = 5
# tags_to_keep = [tag for tag in list(sorted_tags_dict.keys())[:top_k]]
# print(f'Top {top_k} tags are {tags_to_keep}')

# # Remove all the rows where the quote lenght longer than a certain length.

# min_lenth_threshold = 25
# max_length_threshold = 256
# max_len_filtered_dataset = dataset[dataset['quote'].apply(len) <= max_length_threshold].copy()
# min_len_filtered_dataset = max_len_filtered_dataset[max_len_filtered_dataset['quote'].apply(len) >= min_lenth_threshold].copy()

# print(f'After removing all quotes greater than length {max_length_threshold} and lesser than length {min_lenth_threshold}, there are {len(dataset)} rows remaining.')

# min_len_filtered_dataset['tags'] = min_len_filtered_dataset['category'].apply(lambda x: [1 if tag in x.split() else 0 for tag in tags_to_keep])

# # Remove quotes which are not related to any of the chosen tags
# dataset = min_len_filtered_dataset[min_len_filtered_dataset['tags'].apply(lambda x: not all(elem == 0 for elem in x))]
# print(f'After removing all quotes that are not related to top tags, there are {len(dataset)} rows remaining.')

In [ ]:
# seed = 1

# dataset_size = 20000
# sampled_df = dataset.sample(n=dataset_size, random_state=seed).reset_index(drop=True)

# tags_df = pd.DataFrame(sampled_df['tags'].to_list(), columns=tags_to_keep)

# sampled_df = pd.concat([sampled_df, tags_df], axis=1)

# sampled_df = sampled_df.drop(['author','category','tags'], axis=1)
# # sampled_df = sampled_df.drop(['author','category'], axis=1)
# sampled_df.head()

In [ ]:
# sampled_df.sample()

In [ ]:
import pandas as pd
import ast

dataset = pd.read_csv('final_classification.csv', delimiter=',')
dataset = dataset.sample(frac=1).reset_index(drop=True)


dataset['tags_list'] = dataset['tags_list'].apply(ast.literal_eval)
tags_to_keep = ['love', 'life', 'inspirational', 'philosophy', 'humor']

for tag in tags_to_keep:
    dataset[tag] = dataset['tags_list'].apply(lambda x: 1 if tag in x else 0)

print('Shape of dataset : ',dataset.shape)
dataset.head(5)

Shape of dataset :  (29985, 8)


,quote,tags,tags_list,love,life,inspirational,philosophy,humor
0,you can be a dreamer as long as you dont forge...,life_inspirational,"[life, inspirational]",0,1,1,0,0
1,life has pounded me downand thrashed me around...,love_life,"[love, life]",1,1,0,0,0
2,i profess to learn and to teach anatomy not fr...,philosophy,[philosophy],0,0,0,1,0
3,"if your financial life is not in order, every ...",life,[life],0,1,0,0,0
4,reveal to the people and make them experience ...,life,[life],0,1,0,0,0


In [ ]:
def split_data(df, random_seed):

  train_val, test_df = train_test_split(df, test_size=0.2, random_state=random_seed, stratify=df['tags'])
  train_df, val_df = train_test_split(train_val, test_size=0.15, random_state=random_seed, stratify=train_val['tags'])

  for d in [train_df, val_df, test_df]:
    tag_count = {}
    for tag in d['tags_list']:
        for t in tag:
            tag_count[t] = tag_count.get(t, 0) + 1
    print(tag_count)

  train_df = train_df.reset_index()
  val_df = val_df.reset_index()
  test_df = test_df.reset_index()

  return train_df, val_df, test_df

In [ ]:
# train_ratio = 0.7
# test_ratio = 0.15
# valid_ratio = 1 - train_ratio - test_ratio

# # Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
# train_df, temp_df = train_test_split(sampled_df, test_size=test_ratio, random_state=seed)
# valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=seed)

# train_df, test_df, valid_df = train_df.reset_index(drop=True), test_df.reset_index(drop=True), valid_df.reset_index(drop=True)

# print('Training dataset shape: ', train_df.shape)
# print('Testing dataset: shape: ', test_df.shape)
# print('Validation dataset shape: ', valid_df.shape)

train_df, valid_df, test_df = split_data(dataset, 17)

{'humor': 4613, 'life': 7696, 'inspirational': 8161, 'love': 7315, 'philosophy': 6518}
{'inspirational': 1440, 'philosophy': 1150, 'humor': 815, 'love': 1292, 'life': 1359}
{'philosophy': 1916, 'love': 2151, 'life': 2264, 'humor': 1357, 'inspirational': 2400}


In [ ]:
print('Training dataset label proportion:')
print(train_df[tags_to_keep].mean())
print('\nTesting dataset label proportion: ')
print(test_df[tags_to_keep].mean())
print('\nValidation dataset label proportion: ')
print(valid_df[tags_to_keep].mean())

Training dataset label proportion:
love             0.358772
life             0.377458
inspirational    0.400265
philosophy       0.319682
humor            0.226249
dtype: float64

Testing dataset label proportion: 
love             0.358679
life             0.377522
inspirational    0.400200
philosophy       0.319493
humor            0.226280
dtype: float64

Validation dataset label proportion: 
love             0.358989
life             0.377605
inspirational    0.400111
philosophy       0.319533
humor            0.226452
dtype: float64


In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32


EPOCHS = 10
LEARNING_RATE = 2e-05

target_labels = tags_to_keep

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.quote = dataframe.quote
        self.targets = self.data[tags_to_keep].values
        self.max_len = max_len

    def __len__(self):
        return len(self.quote)

    def __getitem__(self, index):
        quote = str(self.quote[index])
        quote = " ".join(quote.split())

        inputs = self.tokenizer.encode_plus(
            quote,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )


        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'quote' : quote
        }

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(valid_df, tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
model = RoBERTaClass()
model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer#, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
import tqdm.notebook as tq
from sklearn.metrics import jaccard_score
def train_model(training_loader, model, optimizer):

    losses = []

    model.train()

    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader),
                      leave=True, colour='steelblue')

    count = 0
    accuracy = 0
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        target_labels = data['targets'].to(device, dtype = torch.float)

        # forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,8)
        loss = loss_fn(outputs, target_labels)
        losses.append(loss.item())

        # training accuracy
        sigmoid_output = torch.sigmoid(outputs)
        label_threshold = 0.5
        predicted_labels = torch.where(sigmoid_output > label_threshold, torch.tensor(1), torch.tensor(0))

        count+=1
        accuracy += jaccard_score(target_labels.cpu(), predicted_labels.cpu(), average='samples')
        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

        # Update progress bar
        loop.set_description(f"")
        loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    return model, float(accuracy)/count, np.mean(losses)

In [ ]:
def eval_model(validation_loader, model, optimizer):
    losses = []
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    count = 0
    accuracy = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            target_labels = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, target_labels)
            losses.append(loss.item())

            # validation accuracy
            sigmoid_output = torch.sigmoid(outputs)
            label_threshold = 0.5
            predicted_labels = torch.where(sigmoid_output > label_threshold, torch.tensor(1), torch.tensor(0))

            count+=1
            accuracy += jaccard_score(target_labels.cpu(), predicted_labels.cpu(), average='samples')

    return float(accuracy)/count, np.mean(losses)

In [ ]:
for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
    val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

    print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')



Epoch 1/10


  0%|          | 0/638 [00:00<?, ?it/s]

train_loss=0.4767, val_loss=0.4565 train_acc=0.5000, val_acc=0.5795
Epoch 2/10


  0%|          | 0/638 [00:00<?, ?it/s]

train_loss=0.4080, val_loss=0.4231 train_acc=0.6163, val_acc=0.6076
Epoch 3/10


  0%|          | 0/638 [00:00<?, ?it/s]

train_loss=0.3568, val_loss=0.4145 train_acc=0.6757, val_acc=0.6292
Epoch 4/10


  0%|          | 0/638 [00:00<?, ?it/s]

train_loss=0.3029, val_loss=0.4358 train_acc=0.7304, val_acc=0.6309
Epoch 5/10


  0%|          | 0/638 [00:00<?, ?it/s]

In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

test_acc, test_loss = eval_model(test_data_loader, model, optimizer)

In [ ]:
test_acc, test_loss

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def get_predictions(model, data_loader):
    model = model.eval()

    quotes = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():
      for data in data_loader:
        # quote = data["quote"]
        ids = data["input_ids"].to(device, dtype = torch.long)
        mask = data["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        target_labels = data["targets"].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        sigmoid_output = torch.sigmoid(outputs)
        label_threshold = 0.5
        predicted_labels = torch.where(sigmoid_output > label_threshold, torch.tensor(1), torch.tensor(0))

        # quotes.extend(quote)
        predictions.extend(predicted_labels)
        prediction_probs.extend(sigmoid_output)
        target_values.extend(target_labels)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    target_values = torch.stack(target_values).cpu()

    return quotes, predictions, prediction_probs, target_values


In [ ]:
quotes, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

In [ ]:
print(classification_report(target_values, predictions, target_names=tags_to_keep))

In [ ]:
import seaborn as sns
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
    plt.ylabel('True category')
    plt.xlabel('Predicted category');

In [ ]:
predictions

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,7)
# cm = confusion_matrix(target_values, predictions)
cm = confusion_matrix(torch.argmax(target_values, dim=1), torch.argmax(predictions, dim=1))
df_cm = pd.DataFrame(cm, index=tags_to_keep, columns=tags_to_keep)
show_confusion_matrix(df_cm)

In [ ]:
test_sample = test_df.sample()
quote = test_sample['quote'].tolist()[0]
target_labels = test_sample[tags_to_keep].values.tolist()[0]

print('Quote :', quote)
print('Tags :', *[tags_to_keep[i] for i in range(len(tags_to_keep)) if target_labels[i]==1], sep=" ")

# quote = 'it is better to doubt that a concept is stupidly flying under your head than profoundly flying over your head.'
encoded_text = tokenizer.encode_plus(
    quote,
    None,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=True,
    padding='max_length',
    return_attention_mask=True,
    truncation=True,
    return_tensors='pt',
)

In [ ]:
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)
token_type_ids = encoded_text['token_type_ids'].to(device)
output = model(input_ids, attention_mask, token_type_ids)
sigmoid_output = torch.sigmoid(output)
label_threshold = 0.5
predicted_labels = torch.where(sigmoid_output > label_threshold, torch.tensor(1), torch.tensor(0)).cpu().tolist()[0]
print('Tags :', *[tags_to_keep[i] for i in range(len(tags_to_keep)) if predicted_labels[i]==1], sep=" ")


In [ ]:
checkpoint_path = 'drive/MyDrive/NLP_Final_Project/checkpoints/model.pt'

checkpoint = {
            'epoch': 10,
            'valid_loss_min': 0.7158,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
# save_ckp(checkpoint, False, checkpoint_path, '')

In [ ]:
# model = DistilBERTClass()
# model.to(device)
# optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# model, optimizer = load_ckp('drive/MyDrive/NLP_Final_Project/checkpoints/model.pt', model, optimizer)

In [ ]:
test_df.iloc[2]

In [ ]:
checkpoint_path = 'drive/MyDrive/NLP_Final_Project/checkpoints/'

seeds = [204]
metrics_df = pd.DataFrame(columns=['Seed', 'Epoch', 'Training_Accuracy', 'Training_Loss', 'Validation_Accuracy', 'Validation_Loss'])

test_metrics_df = pd.DataFrame(columns=['Seed', 'Test_Accuracy', 'Test_Loss'])

for seed in seeds:

    print(f"----------Seed {seed}----------")

    train_df, valid_df, test_df = split_data(dataset, seed)

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
    valid_dataset = CustomDataset(valid_df, tokenizer, MAX_LEN)
    test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    model = RoBERTaClass()
    model.to(device)

    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


    for epoch in range(1, EPOCHS+1):
        print(f'Seed - {seed} , Epoch {epoch}/{EPOCHS}')
        model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
        val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

        print(f'Train Accuracy={train_acc:.4f}, Train Loss={train_acc:.4f}, Validation Accuracy={val_acc:.4f}, Validation Loss={val_loss:.4f}')

        epoch_data = {
            'Seed': seed,
            'Epoch': epoch,
            'Training_Accuracy': train_acc,
            'Training_Loss': train_loss,
            'Validation_Accuracy': val_acc,
            'Validation_Loss': val_loss
        }

        metrics_df.loc[len(metrics_df.index)] = epoch_data

    test_acc, test_loss = eval_model(test_data_loader, model, optimizer)

    test_metrics_data = {
            'Seed': seed,
            'Test_Accuracy': test_acc,
            'Test_Loss': test_loss,
        }
    test_metrics_df.loc[len(test_metrics_df.index)] = test_metrics_data

    print(f'Test Accuracy={test_acc:.4f}, Test Loss={test_loss:.4f}')

    checkpoint = {
        'test_accuracy ': test_acc,
        'test_loss': test_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    save_ckp(checkpoint, False, checkpoint_path + f'distilBert-{seed}.pt', '')

    quotes, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

    with open(f'checkpoint_path'+f'distilBert-{seed}-prediction-data.pkl', 'wb') as f:
        pickle.dump((predictions, prediction_probs, target_values, test_metrics_df, test_metrics_data), f)


----------Seed 204----------
{'inspirational': 8161, 'philosophy': 6518, 'life': 7696, 'love': 7315, 'humor': 4613}
{'love': 1292, 'life': 1359, 'humor': 815, 'philosophy': 1150, 'inspirational': 1440}
{'love': 2151, 'life': 2264, 'inspirational': 2400, 'philosophy': 1916, 'humor': 1357}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Train Accuracy=0.8847, Train Loss=0.8847, Validation Accuracy=0.6359, Validation Loss=0.5949
Seed - 204 , Epoch 9/10


  0%|          | 0/638 [00:00<?, ?it/s]

Train Accuracy=0.9065, Train Loss=0.9065, Validation Accuracy=0.6259, Validation Loss=0.6193
Seed - 204 , Epoch 10/10


  0%|          | 0/638 [00:00<?, ?it/s]

Train Accuracy=0.9237, Train Loss=0.9237, Validation Accuracy=0.6449, Validation Loss=0.6407
Test Accuracy=0.6429, Test Loss=0.6467


In [ ]:
save_ckp(checkpoint, False, checkpoint_path + f'roBERTa-{seed}.pt', '')

quotes, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)

with open(f'checkpoint_path'+f'roBERTa-{seed}-prediction-data.pkl', 'wb') as f:
    pickle.dump((predictions, prediction_probs, target_values, test_metrics_df, test_metrics_data), f)